#### Import Statements

In [ ]:
import sys 
import pdfplumber 
import pandas as pd 
import re 

c:\Users\charl\miniforge3\python.exe


In [ ]:
CATEGORY_KEYWORDS = {
    "Food & Dining": ["chick-fil-a", "starbucks", "mcdonalds", "restaurant"],
    "Groceries": ["walmart", "publix", "kroger", "supermarket"],
    "Gas & Transportation": ["shell", "bp", "chevron", "uber", "lyft"],
    "Subscriptions": ["spotify", "netflix", "hulu", "apple.com/bill"],
}

In [ ]:
### Extract relevant portion of statments document 

pdf_path = 'Documents/_112624 WellsFargo.pdf'

# Extract text from the PDF
with pdfplumber.open(pdf_path) as pdf:
    full_text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())

# Use regex to extract only the transaction history section
cleaned_text = re.search(r"Transaction history(.*?)Monthly service fee summary", full_text, re.DOTALL)

if cleaned_text:
    transactions_text = cleaned_text.group(1)  # Extract the matched section
    print(transactions_text)  # Print to verify
else:
    print("Transaction history section not found")


Check Deposits/ Withdrawals/ Ending daily
Date Number Description Additions Subtractions balance
10/25 Purchase authorized on 10/24 Chevron 0210048 Alpharetta 12.38 8,162.47
GA S304298773774324 Card 5897
10/28 Purchase authorized on 10/25 Chick-Fil-A #01170 Alpharetta 9.03
GA S464299476040552 Card 5897
10/28 Recurring Payment authorized on 10/25 Spotify 877-778-1161 11.99
NY S584299502767203 Card 5897
10/28 Purchase authorized on 10/26 Bp#1994789Capl Roswell GA 32.27
P000000170875353 Card 5897
10/28 Money Transfer authorized on 10/26 Venmo *Madison Ocarro 33.30
VISA Direct NY S304300628807462 Card 5897
10/28 Money Transfer authorized on 10/26 Venmo *Madison Ocarro 11.00
VISA Direct NY S304300628882543 Card 5897
10/28 Purchase authorized on 10/26 Total Wine and Alpharetta GA 18.31
P000000772144318 Card 5897
10/28 Purchase authorized on 10/26 Party City 292 7011 No 9.00
Alpharetta GA P000000280477123 Card 5897
10/28 Purchase authorized on 10/27 Tst*Brooklyn Bagel Johns Creek 11.10
GA S3

In [9]:
# Flag to stop after the first transaction is found
found_transaction = False

# Open the PDF and extract text
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        if found_transaction:  # Stop searching if already found
            break
        
        text = page.extract_text()
        if text:
            lines = text.split("\n")  # Split text into individual lines
            for line in lines:
                # Check if line starts with a date in MM/DD format (detecting transactions)
                if re.match(r"^\d{2}/\d{2}", line):  
                    print("Raw Transaction Line:", line)  # Print the first transaction line found
                    found_transaction = True  # Set flag to stop further searches
                    break  # Stop checking lines on this page

Raw Transaction Line: 10/25 Purchase authorized on 10/24 Chevron 0210048 Alpharetta 12.38 8,162.47


In [ ]:
### Try to print {x} transactions to test parsing progression

pdf_path = 'Documents/_112624 WellsFargo.pdf'
# Counter to track how many transactions we've found
transaction_count = 0
max_transactions = 10  # Set to 2 to capture exactly two transactions

# Open the PDF and extract text
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        if transaction_count >= max_transactions:  # Stop if we've found 2 transactions
            break
        
        text = page.extract_text()
        if text:
            lines = text.split("\n")  # Split text into individual lines
            for line in lines:
                # Check if line starts with a date in MM/DD format (detecting transactions)
                if re.match(r"^\d{2}/\d{2}", line):  
                    print("Raw Transaction Line:", line)  # Print the transaction line
                    transaction_count += 1  # Increment counter
                    if transaction_count >= max_transactions:  # Stop once we hit 2 transactions
                        break  # Stop checking lines on this page


Raw Transaction Line: 10/25 Purchase authorized on 10/24 Chevron 0210048 Alpharetta 12.38 8,162.47
Raw Transaction Line: 10/28 Purchase authorized on 10/25 Chick-Fil-A #01170 Alpharetta 9.03
Raw Transaction Line: 10/28 Recurring Payment authorized on 10/25 Spotify 877-778-1161 11.99
Raw Transaction Line: 10/28 Purchase authorized on 10/26 Bp#1994789Capl Roswell GA 32.27
Raw Transaction Line: 10/28 Money Transfer authorized on 10/26 Venmo *Madison Ocarro 33.30
Raw Transaction Line: 10/28 Money Transfer authorized on 10/26 Venmo *Madison Ocarro 11.00
Raw Transaction Line: 10/28 Purchase authorized on 10/26 Total Wine and Alpharetta GA 18.31
Raw Transaction Line: 10/28 Purchase authorized on 10/26 Party City 292 7011 No 9.00
Raw Transaction Line: 10/28 Purchase authorized on 10/27 Tst*Brooklyn Bagel Johns Creek 11.10
Raw Transaction Line: 10/28 Purchase authorized on 10/27 Tst*Brooklyn Bagel Johns Creek 8.09 8,018.38


### Extract and Parse ALL Transactions

In [23]:
import pdfplumber
import re
import pandas as pd

# Path to the Wells Fargo PDF statement
pdf_path = "Documents/_112624 WellsFargo.pdf"

# Open the PDF and extract full text
print("Opening PDF...\n")
with pdfplumber.open(pdf_path) as pdf:
    full_text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())

# Extract only the transaction history section
match = re.search(r"Transaction history(.*?)Monthly service fee summary", full_text, re.DOTALL)
if not match:
    print("⚠️ ERROR: Could not locate 'Transaction history' section in the document.")
    exit()

transaction_text = match.group(1)
lines = transaction_text.split("\n")

# List to store extracted transactions
transactions = []
current_transaction = None  # Stores multi-line descriptions

# Regex pattern for transaction extraction
pattern = re.compile(
    r"^(\d{2}/\d{2})\s+"  # (1) Capture Date (MM/DD)
    r"(.*?)\s+"  # (2) Capture full description (until numbers)
    r"(-?\d{1,3}(?:,\d{3})*\.\d{2})?\s*"  # (3) Capture deposit/withdrawal if present
    r"(-?\d{1,3}(?:,\d{3})*\.\d{2})?\s*"  # (4) Capture ending balance if present
)

# Processing each line in the extracted transaction history
for line_num, line in enumerate(lines):
    # If the line starts with MM/DD, it's a new transaction
    if re.match(r"^\d{2}/\d{2}", line):  
        if current_transaction:
            # Store the previous transaction before moving to the next one
            transactions.append(current_transaction)

        match = pattern.search(line)
        if match:
            date, description, amount1, amount2 = match.groups()

            # Initialize the new transaction
            current_transaction = {
                "Date": date.strip(),
                "Description": description.strip(),
                "Deposits": None,
                "Withdrawals": None,
                "Balance": None
            }

            # Handling deposits, withdrawals, and balances
            amounts = [amount for amount in [amount1, amount2] if amount]  # Remove None values
            if len(amounts) == 1:
                current_transaction["Balance"] = float(amounts[0].replace(',', ''))
            elif len(amounts) == 2:
                current_transaction["Withdrawals"] = float(amounts[0].replace(',', ''))
                current_transaction["Balance"] = float(amounts[1].replace(',', ''))
    else:
        # Multi-line description handling
        if current_transaction:
            current_transaction["Description"] += " " + line.strip()

# Store the last transaction
if current_transaction:
    transactions.append(current_transaction)

# Convert to DataFrame
df = pd.DataFrame(transactions)

# Convert "Date" to datetime format
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d", errors="coerce")


Opening PDF...



In [24]:
df

,Date,Description,Deposits,Withdrawals,Balance
0,1900-10-25,Purchase GA S304298773774324 Card 5897,None,None,None
1,1900-10-28,Purchase GA S464299476040552 Card 5897,None,None,None
2,1900-10-28,Recurring NY S584299502767203 Card 5897,None,None,None
3,1900-10-28,Purchase P000000170875353 Card 5897,None,None,None
4,1900-10-28,Money VISA Direct NY S304300628807462 Card 5897,None,None,None
5,1900-10-28,Money VISA Direct NY S304300628882543 Card 5897,None,None,None
6,1900-10-28,Purchase P000000772144318 Card 5897,None,None,None
7,1900-10-28,Purchase Alpharetta GA P000000280477123 Card 5897,None,None,None
8,1900-10-28,Purchase GA S304301601188639 Card 5897,None,None,None
9,1900-10-28,Purchase GA S304301617450517 Card 5897,None,None,None
